In [1]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
import matplotlib as mlt
import seaborn as sns

In [2]:
df = pd.read_csv('/home/tommy/Documents/Data Science/Datasets/Sales of summer products and sales in ecommerce for wish.com/summer-products-with-rating-and-performance_2020-08.csv')

In [3]:
#Looking at the data for missing data/outliers
df.info()
#Rating_"number_here"_count all have a missing number of rows -> How to fix?
#

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1573 entries, 0 to 1572
Data columns (total 43 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   title                         1573 non-null   object 
 1   title_orig                    1573 non-null   object 
 2   price                         1573 non-null   float64
 3   retail_price                  1573 non-null   int64  
 4   currency_buyer                1573 non-null   object 
 5   units_sold                    1573 non-null   int64  
 6   uses_ad_boosts                1573 non-null   int64  
 7   rating                        1573 non-null   float64
 8   rating_count                  1573 non-null   int64  
 9   rating_five_count             1528 non-null   float64
 10  rating_four_count             1528 non-null   float64
 11  rating_three_count            1528 non-null   float64
 12  rating_two_count              1528 non-null   float64
 13  rat

In [4]:
df.describe(include = 'all')

,title,title_orig,price,retail_price,currency_buyer,units_sold,uses_ad_boosts,rating,rating_count,rating_five_count,...,merchant_rating_count,merchant_rating,merchant_id,merchant_has_profile_picture,merchant_profile_picture,product_url,product_picture,product_id,theme,crawl_month
count,1573,1573,1573.000000,1573.000000,1573,1573.000000,1573.000000,1573.000000,1573.000000,1528.000000,...,1.573000e+03,1573.000000,1573,1573.000000,226,1573,1573,1573,1573,1573
unique,1201,1203,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,958,NaN,125,1341,1341,1341,1,1
top,Nouvelle mode d'été femmes robe décontractée c...,New Fashion Summer Women Casual Dress Round Ne...,NaN,NaN,EUR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,558c2cdc89d53c4005ea2920,NaN,https://s3-us-west-1.amazonaws.com/sweeper-pro...,https://www.wish.com/c/5e9a74e447f7d92c8db8d14b,https://contestimg.wish.com/api/webimage/5e16c...,5c80e8a150c63d28c67b8f14,summer,2020-08
freq,24,24,NaN,NaN,1573,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,15,NaN,12,3,3,3,1573,1573
mean,NaN,NaN,8.325372,23.288620,NaN,4339.005086,0.432931,3.820896,889.659250,442.263743,...,2.649583e+04,4.032345,NaN,0.143675,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,3.932030,30.357863,NaN,9356.539302,0.495639,0.515374,1983.928834,980.203270,...,7.847446e+04,0.204768,NaN,0.350871,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,1.000000,1.000000,NaN,1.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000e+00,2.333333,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,5.810000,7.000000,NaN,100.000000,0.000000,3.550000,24.000000,12.000000,...,1.987000e+03,3.917353,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,8.000000,10.000000,NaN,1000.000000,0.000000,3.850000,150.000000,79.000000,...,7.936000e+03,4.040650,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,11.000000,26.000000,NaN,5000.000000,1.000000,4.110000,855.000000,413.500000,...,2.456400e+04,4.161797,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Dealing with missing data
#Identifying which column have missing data
columns_missing_data = df.columns[df.isnull().sum() > 0]
#Use .tolist() to turn into a list that can be used to filter out which columns we want to examine further
df_filtered = df[columns_missing_data.to_list()]
#Then use .info() for a summary of those specific columns
df_filtered.info()

#Conclusion: None of these features are ones that I want to include in my model so I will leave them be.
#Second thought: How can rating have 1573 entries yet there are 1528 rows for the individual ratings? 
#Are there some aggregate ratings that were inputted as 0?
df[df["rating"] == 0]
#No, that does not seem to be the case.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1573 entries, 0 to 1572
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   rating_five_count          1528 non-null   float64
 1   rating_four_count          1528 non-null   float64
 2   rating_three_count         1528 non-null   float64
 3   rating_two_count           1528 non-null   float64
 4   rating_one_count           1528 non-null   float64
 5   product_color              1532 non-null   object 
 6   product_variation_size_id  1559 non-null   object 
 7   has_urgency_banner         473 non-null    float64
 8   urgency_text               473 non-null    object 
 9   origin_country             1556 non-null   object 
 10  merchant_name              1569 non-null   object 
 11  merchant_info_subtitle     1572 non-null   object 
 12  merchant_profile_picture   226 non-null    object 
dtypes: float64(6), object(7)
memory usage: 159.9+ KB

,title,title_orig,price,retail_price,currency_buyer,units_sold,uses_ad_boosts,rating,rating_count,rating_five_count,...,merchant_rating_count,merchant_rating,merchant_id,merchant_has_profile_picture,merchant_profile_picture,product_url,product_picture,product_id,theme,crawl_month


In [6]:
#Dealing with the null values in the ratings columns.
#I think dropping the columns could work but I want to try inputing a true value for an equal amount of each rating
#This doesn't make sense to me but I want to try inputting data as practice.

In [7]:
#Detecting outliers in the data
#Plan: Use quantiles to detect outliers for parts of the data where I used visualizations
#Calculating the first and third quantiles of ratings
Q1_rating = np.quantile(df["rating"], .25)
Q3_rating = np.quantile(df["rating"], .75)
#Calculating the interquartile range
IQR_rating = Q3_rating - Q1_rating
#Calculating the lower and upper boundaries for detecting outliers
outlier_lower = Q1_rating-(1.5*IQR_rating)
outlier_higher = Q3_rating+(1.5*IQR_rating)
#Detected outliers
df[(df["rating"] < outlier_lower) | (df["rating"] > outlier_higher)]

,title,title_orig,price,retail_price,currency_buyer,units_sold,uses_ad_boosts,rating,rating_count,rating_five_count,...,merchant_rating_count,merchant_rating,merchant_id,merchant_has_profile_picture,merchant_profile_picture,product_url,product_picture,product_id,theme,crawl_month
5,Plus la taille d'été femmes décontracté sans m...,Plus Size Summer Women Casual Sleeveless Rompe...,3.92,9,EUR,10,0,5.00,1,1.0,...,65,3.507692,5e4b9c3801ba9d210036fc5a,0,NaN,https://www.wish.com/c/5ec645bafd107a02279c8c54,https://contestimg.wish.com/api/webimage/5ec64...,5ec645bafd107a02279c8c54,summer,2020-08
41,Mode féminine été bretelles spaghetti imprimé ...,Women's Summer Fashion Spaghetti Strap Floral ...,7.00,68,EUR,10,0,1.50,2,0.0,...,397,3.858942,5d48eef4b5144c50c2ac0af3,0,NaN,https://www.wish.com/c/5ea69521f5e1e65a3c70faa8,https://contestimg.wish.com/api/webimage/5ea69...,5ea69521f5e1e65a3c70faa8,summer,2020-08
61,Pantalon de couleur unie à la mode pour femmes...,Women s New Fashion Solid Color Pants Loose Ca...,5.00,38,EUR,50,1,5.00,2,2.0,...,9,3.666667,5e4b6a0c57cc9b0b53e499a4,0,NaN,https://www.wish.com/c/5eba6ecb9a9e129fb97b5dfd,https://contestimg.wish.com/api/webimage/5eba6...,5eba6ecb9a9e129fb97b5dfd,summer,2020-08
74,Fashion Women Sleeveless Lace Vest Loose V Nec...,Fashion Women Sleeveless Lace Vest Loose V Nec...,8.00,33,EUR,10,0,5.00,0,NaN,...,1496,4.169786,592955ef980a346d826f7bfc,0,NaN,https://www.wish.com/c/5ed0dfc51fa8446451b19c0d,https://contestimg.wish.com/api/webimage/5ed0d...,5ed0dfc51fa8446451b19c0d,summer,2020-08
110,2020 Femmes Mode Col En V Couleur Unie Slim Fi...,2020 Women Fashion V-neck Solid Color Slim Fit...,5.81,30,EUR,100,0,2.61,18,3.0,...,26473,3.894761,55dc300452a04b658ad68139,0,NaN,https://www.wish.com/c/5e743eded2656b046017c719,https://contestimg.wish.com/api/webimage/5e743...,5e743eded2656b046017c719,summer,2020-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1481,Mode féminine été bretelles spaghetti imprimé ...,Women's Summer Fashion Spaghetti Strap Floral ...,8.00,68,EUR,8,0,5.00,0,NaN,...,626,3.916933,5d3aa36072b0c915fe6b8fbd,0,NaN,https://www.wish.com/c/5eb2200b989caa081980b812,https://contestimg.wish.com/api/webimage/5eb22...,5eb2200b989caa081980b812,summer,2020-08
1488,Nouvelles femmes d'été mode couleur unie short...,New Women Summer Fashion Solid Color Shorts Ca...,14.00,14,EUR,1,0,5.00,0,NaN,...,1134,3.955908,5add589edb5f1f32114d07df,0,NaN,https://www.wish.com/c/5eccb455d4eb5d46b00be0ed,https://contestimg.wish.com/api/webimage/5eccb...,5eccb455d4eb5d46b00be0ed,summer,2020-08
1520,Summer Print Dress Women's Round Neck Short Sl...,Summer Print Dress Women's Round Neck Short Sl...,2.00,2,EUR,100,1,2.44,9,1.0,...,26951,4.035360,58da49c92c3a8f52a755d4a0,0,NaN,https://www.wish.com/c/5eb63e65f98a3634f4d430b3,https://contestimg.wish.com/api/webimage/5eb63...,5eb63e65f98a3634f4d430b3,summer,2020-08
1532,6 Color Plus Size Women's Sexy Lace Patchwork ...,6 Color Plus Size Women's Sexy Lace Patchwork ...,4.00,4,EUR,10,1,5.00,0,NaN,...,6357,4.254208,58b92804935861529969e778,0,NaN,https://www.wish.com/c/5b7d0e51b79c8414c3ef8548,https://contestimg.wish.com/api/webimage/5b7d0...,5b7d0e51b79c8414c3ef8548,summer,2020-08


In [8]:
df[df["rating"] == 0]

,title,title_orig,price,retail_price,currency_buyer,units_sold,uses_ad_boosts,rating,rating_count,rating_five_count,...,merchant_rating_count,merchant_rating,merchant_id,merchant_has_profile_picture,merchant_profile_picture,product_url,product_picture,product_id,theme,crawl_month


In [9]:
lr = LinearRegression()

In [10]:
lr.fit(df[["retail_price"]], df["units_sold"])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [11]:
print(lr.intercept_)

4248.290024106465


In [12]:
print(lr.coef_)

[3.8952527]


In [13]:
#Multiple linear regression
mlr = LinearRegression()
mlr_model = mlr.fit(df[["retail_price", "rating", "countries_shipped_to"]], df["units_sold"])

In [116]:
mlr_intercept = mlr.intercept_

In [117]:
mlr_coefficients = mlr.coef_

In [118]:
#How to evaluate the MLR model?

In [119]:
from sklearn.metrics import r2_score

In [120]:
mlr_coefficients[0] * df["retail_price"]

0        52.814799
1        82.994684
2       162.216882
3        30.179885
4        11.317457
           ...    
1568     33.952371
1569    211.259195
1570     71.677227
1571     41.497342
1572     22.634914
Name: retail_price, Length: 1573, dtype: float64

In [124]:
#Calculating the R2_score
pred_price = (mlr_coefficients[0] * df["retail_price"])
pred_rating = mlr_coefficients[1] * df["rating"]
pred_countries_shipped_to = mlr_coefficients[2] * df["countries_shipped_to"]

y_pred = mlr_intercept + pred_price + pred_rating + pred_countries_shipped_to


In [125]:
#R2 score is terrible
r2_score(df["units_sold"], y_pred)

0.0019453316001892818

0       4306.973166
1       4061.379966
2       4264.075878
3       4428.901317
4       3779.860752
           ...     
1568    4468.909677
1569    4023.774672
1570    4457.534747
1571    3419.019276
1572    4450.345045
Length: 1573, dtype: float64